In [6]:
from src.resolve_init import GetResolve
from pprint import pprint
from dftt_timecode import DfttTimecode

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

DfttTimecode("01:00:00:02", "auto", 24, False, True) > 86420

False